In [ ]:
from bson import ObjectId
from your_db_module import create_mental_health_db
from shared.state import MainState


In [ ]:
def load_memory_node(state: MainState) -> MainState:
    try:
        user_id = state.get("user_id")
        
        if not user_id:
            print("no user_id provided")
            return {
                "past_conversation": [],
                "user_scores": None,
                "user_decay_scores": None,
                "last_update_timestamp": None,
                "calc_result": None
            }
        
        # Convert to ObjectId if string
        if isinstance(user_id, str):
            user_id = ObjectId(user_id)
        
        with create_mental_health_db("mongodb://host.docker.internal:27017/") as db:
            user = db.get_user(user_id)
            
            if user:
                # Load existing user data
                past_conversation = db.get_conversation_history(user_id) or []
                user_scores = db.get_user_scores(user_id)
                user_decay_scores = db.get_decay_scores(user_id)
                
                print(f"loaded data for user: {user_id}")
                
                return {
                    "past_conversation": past_conversation,
                    "user_scores": user_scores,
                    "user_decay_scores": user_decay_scores,
                    "last_update_timestamp": user.get("last_update_timestamp"),
                    "calc_result": user.get("calc_result")
                }
            else:
                # New user
                print(f"new user: {user_id}")
                return {
                    "past_conversation": [],
                    "user_scores": None,
                    "user_decay_scores": None,
                    "last_update_timestamp": None,
                    "calc_result": None
                }
                
    except Exception as e:
        print(f"load failed: {e}")
        return {
            "past_conversation": [],
            "user_scores": None,
            "user_decay_scores": None,
            "last_update_timestamp": None,
            "calc_result": None
        }

In [ ]:

def merge_path_AandB_node(state: MainState) -> MainState:
    # Check what data we have from both paths
    path_a_results = state.get("semantic_search_a_results", [])
    path_b_results = state.get("semantic_search_b_results", [])
    warning_text = state.get("warning_text", "")
    calc_result = state.get("calc_result", 0.0)
    
    # Simple status reporting
    if path_a_results and path_b_results:
        print("merged paths A and B")
    elif path_a_results:
        print("merged path A only")
    elif path_b_results:
        print("merged path B only")
    else:
        print("merged empty paths")
    
    # Simple passthrough - no new fields needed
    # All data already exists in state from both subgraphs
    return {}

In [ ]:
def answer_generator_node(state: MainState) -> MainState:
    try:
        user_query = state.get("user_query", "")
        path_a_results = state.get("semantic_search_a_results", [])
        path_b_results = state.get("semantic_search_b_results", [])
        warning_text = state.get("warning_text", "")
        evaluation_feedback = state.get("evaluation_feedback", "")
        
        # Build context from both paths
        context_sources = []
        
        # Add PATH A results (graded documents)
        for doc in path_a_results:
            context_sources.append(doc.get("text", ""))
        
        # Add PATH B results (search results) 
        for doc in path_b_results:
            context_sources.append(doc.get("text", ""))
        
        # Handle no context case
        if not context_sources:
            print("no context available")
            return {
                "answer": "I understand you're reaching out for support. While I'm experiencing some technical difficulties right now, I want you to know that your concerns are valid. If you're in immediate distress, please contact a mental health professional or crisis helpline. Otherwise, please try again in a few moments."
            }
        
        # Build context text
        context_text = "\n".join([f"- {source[:200]}..." for source in context_sources])
        
        # Add feedback if available
        feedback_section = ""
        if evaluation_feedback:
            feedback_section = f"\n\nImprove based on feedback: {evaluation_feedback}"
        
        # Simplified prompt
        prompt = (
            f"You are a mental health support AI. Provide compassionate, evidence-based guidance.\n\n"
            f"User query: {user_query}\n\n"
            f"Context information:\n{context_text}\n\n"
            f"Assessment: {warning_text if warning_text else 'No specific concerns detected'}{feedback_section}\n\n"
            f"Provide a supportive response (max 300 words) that:\n"
            f"- Addresses the user's query directly\n"
            f"- Uses warm, empathetic language\n"
            f"- Includes practical coping strategies if relevant\n"
            f"- Acknowledges any mental health concerns sensitively\n"
            f"- Reminds user this is supportive information, not professional therapy"
        )
        
        llm_response = llm_model.invoke(prompt)
        answer = llm_response.content.strip()  # Fixed LLM response parsing
        
        print(f"answer generated: {len(answer)} chars")
        
        return {"answer": answer}
        
    except Exception as e:
        print(f"answer generation failed: {e}")
        return {
            "answer": "I understand you're reaching out for support. While I'm experiencing some technical difficulties right now, I want you to know that your concerns are valid. If you're in immediate distress, please contact a mental health professional or crisis helpline. Otherwise, please try again in a few moments."
        }

In [ ]:
MAX_RETRIES = 2

In [ ]:
def evaluator_node(state: MainState) -> MainState:
    try:
        user_query = state.get("user_query", "")
        answer = state.get("answer", "")
        
        # Simple retry tracking (since MainState doesn't have retry counter)
        evaluation_feedback = state.get("evaluation_feedback", "")
        current_attempt = 1 if not evaluation_feedback else 2
        
        prompt = (
            f"Evaluate this mental health AI response (score 0-100):\n\n"
            f"User query: {user_query}\n"
            f"AI response: {answer}\n\n"
            f"Score based on: empathy, safety, relevance, professionalism\n"
            f"Deduct for: medical advice, inappropriate tone, harmful content\n"
            f"If score below 75, provide brief improvement feedback."
        )
        
        llm_response = llm_model.with_structured_output(EvaluationResponse).invoke(prompt)
        
        score = llm_response.score
        feedback = llm_response.feedback if score < 75 else ""
        
        # Decision logic
        if score >= 75:
            print(f"evaluation passed: {score}")
            return {
                "evaluation_result": "ok",
                "evaluation_feedback": ""
            }
        elif current_attempt >= MAX_RETRIES:
            print(f"evaluation failed but max retries reached: {score}")
            return {
                "evaluation_result": "ok",  # Accept to avoid infinite loop
                "evaluation_feedback": ""
            }
        else:
            print(f"evaluation failed, retry: {score}")
            return {
                "evaluation_result": "Not ok",
                "evaluation_feedback": feedback
            }
            
    except Exception as e:
        print(f"evaluation failed: {e}")
        return {
            "evaluation_result": "ok",  # Fail-safe to continue
            "evaluation_feedback": ""
        }

In [ ]:
# Import needed
from bson import ObjectId
from your_db_module import create_mental_health_db

def save_memory_node(state: MainState) -> MainState:
    try:
        user_id = state.get("user_id")
        user_query = state.get("user_query", "")
        answer = state.get("answer", "")
        
        if not user_id:
            print("no user_id to save")
            return {}
        
        # Convert to ObjectId if string
        if isinstance(user_id, str):
            user_id = ObjectId(user_id)
        
        with create_mental_health_db("mongodb://host.docker.internal:27017/") as db:
            # Save conversation (APPEND pattern)
            conversation_saved = db.append_conversation(user_id, user_query, answer)
            
            # Prepare data for bulk update (OVERWRITE pattern)  
            update_data = {}
            if state.get("user_scores") is not None:
                update_data["user_scores"] = state.get("user_scores")
            if state.get("user_decay_scores") is not None:
                update_data["user_decay_scores"] = state.get("user_decay_scores") 
            if state.get("last_update_timestamp") is not None:
                update_data["last_update_timestamp"] = state.get("last_update_timestamp")
            if state.get("calc_result") is not None:
                update_data["calc_result"] = state.get("calc_result")
            
            # Save metrics data
            metrics_saved = db.bulk_update_user(user_id, update_data) if update_data else True
            
            if conversation_saved and metrics_saved:
                print(f"memory saved for user: {user_id}")
            else:
                print(f"partial save for user: {user_id}")
            
            return {}  # Simple completion - no new state fields needed
            
    except Exception as e:
        print(f"memory save failed: {e}")
        return {}  # Continue graph execution even on save failure